In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.basics import *
from datetime import datetime
from functools import partial

In [50]:
path=Path('data')
df=pd.read_csv(path/'datacsv.csv')
df

,Date,ambient temperature | (C),wind speed (m/s),Irradiance DHI (W/m2),Irradiance DNI(W/m2),Sun altitude angle | (deg),Sun azimuth angle(deg),DC power(kW),Operating DC voltage(V)
0,"Jan 1, 12:00 am",13,0.3,0,0,-75.8320,41.9988,0.0,0.0
1,"Jan 1, 01:00 am",13,0.4,0,0,-65.2542,70.7090,0.0,0.0
2,"Jan 1, 02:00 am",13,0.4,0,0,-53.0596,83.7640,0.0,0.0
3,"Jan 1, 03:00 am",13,0.4,0,0,-40.5757,92.5174,0.0,0.0
4,"Jan 1, 04:00 am",14,0.4,0,0,-28.1574,99.8838,0.0,0.0
...,...,...,...,...,...,...,...,...,...
429235,"Dec 31, 07:00 pm",-12,2.8,0,0,-32.0172,267.7870,0.0,0.0
429236,"Dec 31, 08:00 pm",-13,2.8,0,0,-42.9419,278.5310,0.0,0.0
429237,"Dec 31, 09:00 pm",-13,2.5,0,0,-53.5005,292.1140,0.0,0.0
429238,"Dec 31, 10:00 pm",-14,2.3,0,0,-62.8139,311.9830,0.0,0.0


In [51]:
df['Date']=pd.to_datetime(df['Date'],format="%b %d,  %I:%M %p")

In [52]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
#     targ_pre = re.sub('[Dd]ate$', '','mew')
    attr = ['Month', 'Day']
    if time: attr = attr + ['Hour']
    for n in attr: df[n] = getattr(fld.dt, n.lower())
#     df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [53]:
add_datepart(df,'Date',time=True)

In [55]:
df.head(20)

,ambient temperature | (C),wind speed (m/s),Irradiance DHI (W/m2),Irradiance DNI(W/m2),Sun altitude angle | (deg),Sun azimuth angle(deg),DC power(kW),Operating DC voltage(V),Month,Day,Hour
0,13,0.3,0,0,-75.83200,41.9988,0.000000,0.0000,1,1,0
1,13,0.4,0,0,-65.25420,70.7090,0.000000,0.0000,1,1,1
2,13,0.4,0,0,-53.05960,83.7640,0.000000,0.0000,1,1,2
3,13,0.4,0,0,-40.57570,92.5174,0.000000,0.0000,1,1,3
4,14,0.4,0,0,-28.15740,99.8838,0.000000,0.0000,1,1,4
5,14,0.4,0,0,-16.00520,106.9810,0.000000,0.0000,1,1,5
6,14,0.4,0,0,-4.31602,114.4550,0.000000,0.0000,1,1,6
7,14,0.4,2,0,6.23275,122.8320,0.000000,30.0000,1,1,7
8,15,0.3,5,0,16.04580,132.6430,0.000000,30.0000,1,1,8
9,15,0.2,28,0,24.32370,144.4020,0.004690,30.0000,1,1,9


In [56]:
df.to_pickle('data/data_clean')